# Tutorial

Intake-thredds provides an interface that combines [`siphon`](https://github.com/Unidata/siphon) and `intake` to retrieve data from THREDDS data servers. This tutorial provides an introduction to the API and features of intake-thredds. Let's begin by importing `intake`. 

In [1]:
import intake

## Loading a catalog

You can load data from a THREDDS catalog by providing the URL to a valid THREDDS catalog: 

In [2]:
cat_url = 'https://psl.noaa.gov/thredds/catalog/Datasets/noaa.ersst/catalog.xml'

In [3]:
catalog = intake.open_thredds_cat(cat_url)
print(catalog)
print(type(catalog))

<Intake catalog: No name found>
<class 'intake_thredds.cat.ThreddsCatalog'>


## Using the catalog

Once you've loaded a catalog, you can display its contents by iterating over its entries:

In [4]:
list(catalog)

['err.mnmean.v3.nc',
 'sst.mnmean.v3.nc',
 'sst.mnmean.v4.nc',
 'sst.mon.1971-2000.ltm.v4.nc',
 'sst.mon.19712000.ltm.v3.nc',
 'sst.mon.1981-2010.ltm.v3.nc',
 'sst.mon.1981-2010.ltm.v4.nc']

Once you've identified a dataset of interest, you can access it as follows:

In [5]:
source = catalog['err.mnmean.v3.nc']
print(source)

sources:
  err.mnmean.v3.nc:
    args:
      chunks: {}
      urlpath: https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst/err.mnmean.v3.nc
    description: THREDDS data
    driver: intake_xarray.opendap.OpenDapSource
    metadata:
      catalog_dir: null



In [6]:
print(type(source))

<class 'intake_xarray.opendap.OpenDapSource'>


## Loading a dataset

To load a dataset of interest, you can use the `to_dask()` method which is available on a **source** object:

In [7]:
%%time
ds = source().to_dask()
ds

CPU times: user 719 ms, sys: 185 ms, total: 904 ms
Wall time: 9.44 s


<xarray.Dataset>
Dimensions:    (lat: 89, lon: 180, nbnds: 2, time: 1994)
Coordinates:
  * lat        (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon        (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time       (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2020-02-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(1994, 2), meta=np.ndarray>
    err        (time, lat, lon) float32 dask.array<chunksize=(1994, 89, 180), meta=np.ndarray>
Attributes:
    title:                           NOAA Extended Reconstructed SST V3
    Conventions:                     CF-1.0
    history:                         Thu Jul  1 14:04:15 2010: ncatted -O -a ...
    comments:                        The extended reconstructed sea surface t...
    platform:                        Model
    source:                          NOAA/NESDIS/National Climatic Data Center
    institution:                     NOAA/NESDIS/National Climatic Data Center
    citation:                        Smith, T.M., R.W. Reynolds, Thomas C. Pe...
    dataset_title:                   Extended Reconstructed Sea Surface Tempe...
    source_doc:                      https://www.ncdc.noaa.gov/data-access/ma...
    References:                      https://www.psl.noaa.gov/data/gridded/da...
    DODS_EXTRA.Unlimited_Dimension:  time

The `to_dask()` reads only metadata needed to construct an ``xarray.Dataset``. The actual data are streamed over the network when computation routines are invoked on the dataset. 
By default, intake-thredds uses ``chunks={}`` to load the dataset with dask using a single chunk for all arrays. You can use a different chunking scheme by prividing a custom value of chunks before calling `.to_dask()`:

In [8]:
%%time
# Use a custom chunking scheme
ds = source(chunks={'time': 100, 'lon': 90}).to_dask()
ds

CPU times: user 218 ms, sys: 18.3 ms, total: 237 ms
Wall time: 8.3 s


<xarray.Dataset>
Dimensions:    (lat: 89, lon: 180, nbnds: 2, time: 1994)
Coordinates:
  * lat        (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon        (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time       (time) datetime64[ns] 1854-01-01 1854-02-01 ... 2020-02-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 dask.array<chunksize=(100, 2), meta=np.ndarray>
    err        (time, lat, lon) float32 dask.array<chunksize=(100, 89, 90), meta=np.ndarray>
Attributes:
    title:                           NOAA Extended Reconstructed SST V3
    Conventions:                     CF-1.0
    history:                         Thu Jul  1 14:04:15 2010: ncatted -O -a ...
    comments:                        The extended reconstructed sea surface t...
    platform:                        Model
    source:                          NOAA/NESDIS/National Climatic Data Center
    institution:                     NOAA/NESDIS/National Climatic Data Center
    citation:                        Smith, T.M., R.W. Reynolds, Thomas C. Pe...
    dataset_title:                   Extended Reconstructed Sea Surface Tempe...
    source_doc:                      https://www.ncdc.noaa.gov/data-access/ma...
    References:                      https://www.psl.noaa.gov/data/gridded/da...
    DODS_EXTRA.Unlimited_Dimension:  time